<h1>HW4 - Attention and Transformers- Practical Q2</h1>
<h3><font color=yellow>Total Points: 100</font></h3>



<font size=+4>
Introduction
</font>
<br/>
<font size=+2>

<br/>
Part-of-speech tagging (POS tagging) is a natural language processing task where each word in a given text is assigned a specific part-of-speech category, such as noun, verb, adjective, etc. In the context of Roberta, a state-of-the-art language model based on the transformer architecture, POS tagging involves leveraging the model's pre-trained knowledge to accurately predict and label the grammatical roles of words in a sentence. Roberta's extensive training data and attention mechanisms enable it to capture subtle contextual cues, allowing for more nuanced and accurate POS tagging. By understanding the syntactic structure of a sentence, Roberta enhances the efficiency and precision of POS tagging, contributing to more sophisticated language understanding and downstream applications in natural language processing.
</font>


<font size=+2>
In this section we want to fine-tune the Roberta that you trained in previous section for POS-Tagging Task on Parsig dataset
<font/>

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate
!pip install seqeval

import re
import torch
import pandas as pd

import numpy as np
import transformers
import datasets
import matplotlib.pyplot as plt


In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# All label that exist
# AAX mean Unknown used for CLS and other symbols
tags = {'AAX', 'B-ADJ', 'I-ADJ', 'B-ADV', 'I-ADV', 'B-N', 'I-N', 'B-V', 'I-V',\
        'B-PRONOUN', 'I-PRONOUN', 'B-NUM', 'I-NUM', 'B-DET', 'I-DET', 'B-PRE',\
        'I-PRE', 'B-POST', 'I-POST', 'B-CONJ', 'I-CONJ', 'B-JUNK', 'I-JUNK',\
        'B-MARKER', 'I-MARKER', }
# Convert lables to speceif numeric id
tag2id = {tag: id for id, tag in enumerate(sorted(list(tags)))}
id2tag = {id: tag for tag, id in tag2id.items()}

In [3]:
######################   TODO 1.1   ########################
# Load pretrained model that trained in previous sectiopn
# Load tokenizer from previous section
# Freeze base model for fine-tuning
###################### (5 points) ##########################
model_name = "Amir01/Roberta-finetuned-Parsi-Mianeh-Corpus"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForTokenClassification.from_pretrained(model_name, num_labels=25, id2label=id2tag, label2id=tag2id)

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/65.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/291k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at Amir01/Roberta-finetuned-Parsi-Mianeh-Corpus and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
######################   TODO 1.2  ########################
# Load datasets
###################### (5 points) ##########################
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [5]:
######################   TODO 1.3   ########################
# Complete custom data set
# Use Torch dataloader for datasets
###################### (15 points) ##########################
class CustomDataset:
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        row = self.df.iloc[index, :]
        text = row['text']
        labels = row['labels']
        tokenized_text = self.tokenizer(text, truncation=True)
        labels_id = [tag2id[tag] for tag in labels.split(',')]
        labels_id = [tag2id['AAX']] + labels_id + [tag2id['AAX']]
        tokenized_text['labels'] = labels_id
        return tokenized_text

    def __len__(self):
        return len(self.df)

In [6]:
train_dataset = CustomDataset(train_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
class Learning_Rate_Scheduler():
    '''A simple wrapper class for learning rate scheduling'''
    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [ ]:
######################   TODO 2.1   ########################
# Write a code for training model with training datset
# At each epoch report accurcy for validation dataset
# Save best model by accuracy
###################### (30 points) ##########################
from tqdm import tqdm
import torch.nn.functional as F

class Trainer:
    def __init__(
        self,
        model,
        train_dataloader,
        num_epochs,
        test_dataloader=None,
        lr= 1e-4,
        weight_decay=0.08,
        betas=(0.9, 0.999),
        warmup_steps=10000,
        device='cuda'
        ):

        self.device = device
        self.model = model
        self.train_data = train_dataloader
        self.test_data = test_dataloader


        """ Defining optimizer and Learning Rate scheduler """
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.lr_scheduler = Learning_Rate_Scheduler(
            self.optimizer, self.model.d_model, n_warmup_steps=warmup_steps
            )

        """ Using CrossEntropy Loss function for predicting the masked_token """
        self.criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))



    def Fit(self, train_loader, num_epochs, path_to_save):

      self.train_losses = []
      lrs=[]

      for epoch in range(num_epochs):
        train_per_epoch_loss=0
        train_data_total = 0

        with tqdm(train_loader, unit="batch") as batches:
          for data in batches:
            batches.set_description(f"Epoch {epoch + 1}")

            # 0. batch_data will be sent into the device(GPU or cpu)
            data = {key: value.to(self.device) for key, value in data.items()}

            #################################################################################
            #                          COMPLETE THE FOLLOWING SECTION                       #
            #################################################################################
            # 1. forward phase
            # Your Code Here
            output = self.model(data['input_ids'], data['segment_ids'])
            target = data['labels']


            # 2. calculate Loss only for MASKed tokens
            """ Attention! non-masked token ids are set to -100 """
            # Your Code Here
            loss = self.criterion(output.view(-1, output.size(-1)), target.view(-1))

            #################################################################################
            #                                """ 7 points """
            #################################################################################
            # 3. backward and optimization
            self.lr_scheduler.zero_grad()
            loss.backward()
            self.lr_scheduler.step_and_update_lr()

            lrs.append(self.optimizer.param_groups[0]["lr"])
            train_data_total += len(data["labels"])
            train_per_epoch_loss += loss.item()
            batches.set_postfix(train_loss = (train_per_epoch_loss/len(train_loader))) # update loss per batch of data

        self.train_losses.append(train_per_epoch_loss/len(train_loader))

        if epoch%10==0:
          PATH=f"{path_to_save}mlm_{num_epochs}Eps_{epoch}"

          torch.save({
                      'epoch': epoch,
                      'model_state_dict': self.model.state_dict(),
                      'optimizer_state_dict': self.optimizer.state_dict(),
                      'loss': self.train_losses[-1],
                      }, PATH)

      self.history={
        'train_losses':self.train_losses,
        'optimizer':self.optimizer,
        'learning_rates':lrs
      }
      return self.history

In [ ]:
######################   TODO 2.2   ########################
# Report best model accuarcy on validation dataset
# Accuracy below 85% will not be graded
###################### (5 points) ##########################

In [ ]:
#logging in to hugging face
from huggingface_hub import notebook_login

notebook_login()

In [7]:
train_dataset_hf = datasets.Dataset.from_list(train_dataset)
test_dataset_hf = datasets.Dataset.from_list(test_dataset)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
######################   TODO 2.1   ########################
# Now implement it with huggingface trainer
###################### (40 points) ##########################
import evaluate
import numpy as np


seqeval = evaluate.load('seqeval')

data_collator = transformers.DataCollatorForTokenClassification(tokenizer=tokenizer)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

training_args = transformers.TrainingArguments(
    output_dir="roberta-pos-tagging",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    # weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_hf,
    eval_dataset=test_dataset_hf,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.698960,0.715487,0.801864,0.756217,0.770114
2,No log,0.611950,0.742988,0.832685,0.785283,0.802858
3,0.748500,0.552818,0.745068,0.839136,0.789309,0.806600
4,0.748500,0.528944,0.766617,0.847942,0.805231,0.822759
5,0.424900,0.493621,0.771752,0.851014,0.809447,0.830413
6,0.424900,0.507275,0.784722,0.856236,0.818921,0.836962
7,0.424900,0.490667,0.785041,0.853369,0.817780,0.835176
8,0.299400,0.488136,0.798614,0.861663,0.828942,0.843086
9,0.299400,0.501332,0.803163,0.863199,0.832099,0.844957
10,0.213300,0.486543,0.809697,0.868728,0.838174,0.853121


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AAX seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AAX seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AAX seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AAX seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AAX seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: Use

TrainOutput(global_step=3075, training_loss=0.3186858302790944, metrics={'train_runtime': 1036.881, 'train_samples_per_second': 47.349, 'train_steps_per_second': 2.966, 'total_flos': 2281969697466900.0, 'train_loss': 0.3186858302790944, 'epoch': 15.0})